In [1]:
import numpy as np
from quaos.symplectic import PauliSum, PauliString, Pauli, Xnd, Ynd, Znd, Id, string_to_symplectic, symplectic_to_string
from quaos.gates import GateOperation, Circuit, Hadamard as H, SUM as CX, PHASE as S
from quaos.hamiltonian import *
from quaos.core.prime_Functions_Andrew import int_to_bases
from collections import defaultdict
import sympy as sym
from sympy.physics.quantum import TensorProduct,Operator

In [45]:
def xi(a, d):
    """
    Computes the a-th eigenvalue of a pauli with dimension d.
    
    Parameters:
        a (int): The integer to compute the eigenvalue for.
        d (int): The dimension of the pauli to use.
    
    Returns:
        complex: The computed eigenvalue.
    """
    return np.exp(2 * np.pi * 1j * a / d)

def group_indices(lst):
    """
    Groups indices of the same value in a list into sublists.
    For example, if the input list is [1, 2, 1, 3, 2], the output will be [[0, 2], [1, 4], [3]].
    """
    index_dict = defaultdict(list)
    for idx, value in enumerate(lst):
        index_dict[value].append(idx)
    
    return [indices for indices in index_dict.values()]


def Hadamard_Symmetric_PauliSum(n_paulis,n_qubits,n_sym_q):
    # create coefficients
    c_int_bands = np.sort(np.random.randint(n_paulis,size=n_paulis))
    c_bands = group_indices(c_int_bands)

    coefficients = np.zeros(n_paulis)
    sym_bands = []
    for i,b in enumerate(c_bands):
        coefficients[b] = np.random.normal(0, 1)
        if len(b) != 1:
            sym_bands.append(b)

    #print(sym_bands)
    n_extra_bands = np.floor(np.sum([len(b) for b in sym_bands])/2 - n_sym_q)
    pauli_strings = ['' for i in range(n_paulis)]

    all_x = []
    all_z = []
    for i in range(n_sym_q):
        x_pauli = []
        z_pauli = []
        if len(sym_bands) >= 1:
            b_ind = np.random.randint(len(sym_bands))
            b = sym_bands[b_ind]
            x_ind = np.random.randint(len(b))
            x_pauli.append(b[x_ind])
            b.pop(x_ind)
            z_ind = np.random.randint(len(b))
            z_pauli.append(b[z_ind])
            b.pop(z_ind)
            if len(b) < 2:
                sym_bands.pop(b_ind)
            else:
                sym_bands[b_ind] = b

            # randomly adding extra x and zs if possible
            if n_extra_bands > 0 and len(sym_bands) >= 1:
                extras = np.random.randint(n_extra_bands)
                n_extra_bands -= extras
                for j in range(extras):
                    b_ind = np.random.randint(len(sym_bands))
                    b = sym_bands[b_ind]
                    x_ind = np.random.randint(len(b))
                    x_pauli.append(b[x_ind])
                    b.pop(x_ind)
                    z_ind = np.random.randint(len(b))
                    z_pauli.append(b[z_ind])
                    b.pop(z_ind)
                    if len(b) < 2:
                        sym_bands.pop(b_ind)
                    else:
                        sym_bands[b_ind] = b
        print(coefficients[x_pauli])
        for j in range(n_paulis):
            if j in x_pauli:
                pauli_strings[j] += 'x1z0 '
            elif j in z_pauli:
                pauli_strings[j] += 'x0z1 '
            else:
                pauli_strings[j] += 'x0z0 '
        all_x += x_pauli
        all_z += z_pauli
    print(all_x,all_z)
    non_sym_paulis = [i for i in range(n_paulis) if not i in all_x and not i in all_z]
    q_dims = [2 for i in range(2*(n_qubits-n_sym_q))]
    available_paulis = list(np.arange(int(np.prod(q_dims))))
    for i,x in enumerate(all_x):
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[x] += f"x{r}z{s} "
            pauli_strings[all_z[i]] += f"x{r}z{s} "

        pauli_strings[x].strip()
        pauli_strings[all_z[i]].strip()

    for i in non_sym_paulis:
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[i] += f"x{r}z{s} "
        pauli_strings[i].strip()

    P = PauliSum(pauli_strings, weights=coefficients ,dimensions=[2 for i in range(n_qubits)], phases=None,standardise=False)
    #print(P)

    # construct random Clifford circuit
    C = Circuit(dimensions=[2 for i in range(n_qubits)])
    gate_list = [H,S,CX]
    gg = []
    for i in range(100):
        g_i = np.random.randint(3)
        if g_i == 2:
            aa = list(random.sample(range(n_qubits), 2))
            gg += [gate_list[g_i](aa[0],aa[1],2)]
            #print(aa)
        else:
            aa = list(random.sample(range(n_qubits), 1))
            gg += [gate_list[g_i](aa[0],2)]
        
    C.add_gate(gg)
    P = C.act(P)

    phases = P.phases
    cc = P.weights
    ss = P.pauli_strings
    dims = P.dimensions

    cc *= np.array([-1]*n_paulis)**phases
    P = PauliSum(ss, weights=cc ,dimensions=dims, phases=None,standardise=False)

    # qubits shuffle qubits (Fisher Yates Shuffle)
    '''
    gg = []
    order = np.arange(n_qubits)
    for i in np.arange(n_qubits)[::-1]:
        j = np.random.randint(i+1)
        gg += [GateOperation('SWAP',(i,j),['x1z0 x0z0 -> x0z0 x1z0', 'x0z1 x0z0 -> x0z0 x0z1', 'x0z0 x1z0 -> x1z0 x0z0', 'x0z0 x0z1 -> x0z1 x0z0'],2)]
        a = int(order[i])
        b = int(order[j])
        order[i] = b
        order[j] = a
        
    for g in gg:
        P = g.act(P)
    sym_qubit_ind = []
    for i in range(n_qubits):
        if order[i] in np.arange(n_sym_q):
            sym_qubit_ind.append(i)
    '''
    #print('Symmetric qubits: ',sym_qubit_ind)
    #print(P)
    return(P,C)

def add_s2(P,cq,qubit):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),H(qubit,2),CX(qubit,cq,2),S(qubit,2),H(qubit,2)])
    return(C)

def add_r2(P,cq,qubit):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(cq,2),CX(qubit,cq,2),S(cq,2)])
    return(C)

def add_r2s2(P,cq,qubit):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),CX(cq,qubit,2),H(qubit,2),CX(qubit,cq,2),S(qubit,2),H(qubit,2)])
    return(C)

def number_of_I(P,Pauli_index,qubits):
    """
    Returns the number of I's in the Pauli string at Pauli_index for qubits.
    """

    return np.sum([1 for i in qubits if P.x_exp[Pauli_index,i] == 0 and P.z_exp[Pauli_index,i] == 0])

def flatten_list(nested_list):
    """
    Flattens a list of lists into a single list with all the elements.
    
    Args:
        nested_list (list of lists): The input list of lists.
    
    Returns:
        list: A single flattened list containing all elements.
    """
    return [item for sublist in nested_list for item in sublist]

def find_circuit(start_pauli,goal_pauli,iterations,compare_phases = False):
    n_qudits = start_pauli.n_qudits()
    SUMs = [CX(i, j, 2) for i in range(n_qudits) for j in range(n_qudits) if i != j]
    Ss = [S(i, 2) for i in range(n_qudits)]
    Hs = [H(i, 2) for i in range(n_qudits)]
    all_gates = SUMs + Ss + Hs

    goal_circuits = []
    circuits = [Circuit(dimensions=[2 for i in range(n_qudits)])]
    intermediate_paulis = [start_pauli.copy()]
    for i in range(iterations):
        print(i,len(intermediate_paulis),len(goal_circuits))
        intermediate_paulis_old = intermediate_paulis.copy()
        for ip,p in enumerate(intermediate_paulis_old):
            for g in all_gates:
                P_temp = g.act(p)
                if not compare_phases:  
                    P_temp.phases = [0] 
                C_temp = Circuit(dimensions=[2 for i in range(n_qudits)])
                for g2 in circuits[ip].gates:
                    C_temp.add_gate(g2)
                C_temp.add_gate(g)
                if P_temp not in intermediate_paulis:
                    intermediate_paulis.append(P_temp)
                    circuits.append(C_temp)
                if P_temp == goal_pauli:
                    if C_temp not in goal_circuits:
                        goal_circuits.append(C_temp)
                    #print("Found goal circuit") 
    return(goal_circuits)

def find_circuit_simultaneous(start_paulis,goal_paulis,iterations,compare_phases = True):
    n_qudits = start_paulis[0].n_qudits()
    SUMs = [CX(i, j, 2) for i in range(n_qudits) for j in range(n_qudits) if i != j]
    Ss = [S(i, 2) for i in range(n_qudits)]
    Hs = [H(i, 2) for i in range(n_qudits)]
    all_gates = SUMs + Ss + Hs

    goal_circuits = []
    circuits = [[Circuit(dimensions=[2 for i in range(n_qudits)])] for _ in range(len(start_paulis))]
    intermediate_paulis = [[start_paulis[i]] for i in range(len(start_paulis))]
    
    for i in range(iterations):
        intermediate_paulis_new = [[] for _ in range(len(start_paulis))]
        circuits_new = [[] for _ in range(len(start_paulis))]
        for g in all_gates:
            for ip in range(len(intermediate_paulis[0])):
                current_paulis = []
                for ib,b in enumerate(intermediate_paulis):
                    p = b[ip]
                    P_temp = g.act(p)
                    if not compare_phases:
                        P_temp.phases = [0]
                    C_temp = Circuit(dimensions=[2 for i in range(n_qudits)])
                    for g2 in circuits[ib][ip].gates:
                        C_temp.add_gate(g2)
                    C_temp.add_gate(g)
                    intermediate_paulis_new[ib].append(P_temp)
                    circuits_new[ib].append(C_temp)
                    current_paulis.append(P_temp)
                if all([current_paulis[j] in goal_paulis for j in range(len(current_paulis))]):
                    goal_circuits.append(circuits[ib][ip])
                    return(C_temp)
                    #print("Found goal circuit")
        intermediate_paulis = intermediate_paulis_new.copy()
        circuits = circuits_new
    return(None)

def modulo_2(expr):
    """
    Takes a SymPy expression and reduces its coefficients modulo 2.
    """
    # Expand the expression to handle all terms
    expr = expr.expand()
    
    # Iterate through the terms and apply modulo 2 to coefficients
    terms = expr.as_ordered_terms()
    mod_expr = sum(sym.Mod(term.as_coeff_Mul()[0], 2) * term.as_coeff_Mul()[1] for term in terms)
    
    return mod_expr

def reduce_exponents(expr):
    """
    Reduces all exponents in a SymPy expression to zero, assuming symbols are binary (0 or 1).
    
    Args:
        expr (sympy.Expr): The input SymPy expression.
    
    Returns:
        sympy.Expr: The modified expression with all exponents set to zero.
    """
    expr = sym.expand(expr)  # Expand the expression to handle all terms
    return expr.replace(lambda x: x.is_Pow, lambda x: x.base)

def symplectic_effect(circuit,phases = True):
    n_qudits = len(circuit.dimensions)
    #print(n_qudits)
    r_now = list(sym.symbols([f'r{i}' for i in range(1, n_qudits+1)]))
    omega = sym.symbols('omega')
    #print(len(r_now))
    s_now = list(sym.symbols([f's{i}' for i in range(1, n_qudits+1)]))
    r_next = [r_now[i] for i in range(n_qudits)]
    s_next = [s_now[i] for i in range(n_qudits)]

    #r1_start, r2_start, s1_start, s2_start = sym.symbols('r1 r2 s1 s2')
    X = Operator('X')
    Z = Operator('Z')

    phase = 0
    gates = circuit.gates
    qubits = circuit.indexes
    for i,g in enumerate(gates):
        if g.name == 'SUM':
            r_next[qubits[i][1]] = r_now[qubits[i][0]] + r_now[qubits[i][1]]
            s_next[qubits[i][0]] = s_now[qubits[i][0]] + s_now[qubits[i][1]]
            #phase += r_now[qubits[i][1]] * s_now[qubits[i][1]] * (r_now[qubits[i][0]] + s_now[qubits[i][0]] + 1)
        elif g.name == 'H' or g.name == 'HADAMARD':
            r_next[qubits[i][0]] = s_now[qubits[i][0]]
            s_next[qubits[i][0]] = r_now[qubits[i][0]]
            phase += s_now[qubits[i][0]] * r_now[qubits[i][0]]
        elif g.name == 'S' or g.name == 'PHASE':
            s_next[qubits[i][0]] = s_now[qubits[i][0]] + r_now[qubits[i][0]]
            #phase += s_now[qubits[i][0]] * r_now[qubits[i][0]]
        r_now = [r_next[i] for i in range(n_qudits)]
        s_now = [s_next[i] for i in range(n_qudits)]
    final = TensorProduct(X**(modulo_2(r_now[0])) * Z**(modulo_2(s_now[0])), X**(modulo_2(r_now[1])) * Z**(modulo_2(s_now[1])))
    for i in range(2, n_qudits):
        final = TensorProduct(final, X**(modulo_2(r_now[i])) * Z**(modulo_2(s_now[i])))
    
    if phases:
        display(omega**modulo_2(reduce_exponents(modulo_2(sym.simplify(phase)))) * final)
    else:
        display(final)
    return()



In [46]:
def add_phase(P,cq,qubit,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),CX(cq,qubit,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit),S(cq,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)     

In [ ]:
def prepare_sym_candidates(P1,pi,pj,C,current_qubit):
    cq = current_qubit
    # prepare anti-commuting pauli strings with the same absolute coefficients for test of hadamard Symmetry
    # prime pauli pi and pj for cancel_pauli
    if P1.x_exp[pi, cq] == 1 and P1.z_exp[pj, cq] == 1: # x,z
        px = pi
        pz = pj
    elif P1.z_exp[pi, cq] == 1 and P1.x_exp[pj, cq] == 1: # z,x
        px = pj
        pz = pi
    elif P1.x_exp[pi, cq] == 1 and P1.z_exp[pj, cq] == 0: # x,id or x,x
        if any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        elif any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.z_exp[pi, cq] == 1 and P1.x_exp[pj, cq] == 0: # z,id or z,z
        if any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
        elif any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pj
        pz = pi
    elif P1.x_exp[pi, cq] == 0 and P1.z_exp[pj, cq] == 1: # id,z
        if any(P1.x_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
        elif any(P1.z_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.x_exp[pi, cq] == 0 and P1.x_exp[pj, cq] == 1: # id,x
        if any(P1.z_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        elif any(P1.x_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pj
        pz = pi
    else: # id,id
        if any(P1.x_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            elif any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = H(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pi
            pz = pj
        elif any(P1.z_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
            elif any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = H(min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pj
            pz = pi
    return(C, P1, px, pz)

def check_current_paulis(P1,pi,pj,current_qubit,cc_bands):
    current_qubit_pauli_check = True
    for i in range(P1.n_qudits()):
        if i != pi and i != pj:
            if P1.x_exp[i, current_qubit] == 1 and P1.z_exp[i, current_qubit] == 1: # Y
                continue
            elif P1.x_exp[i, current_qubit] == 0 and P1.z_exp[i, current_qubit] == 0: # I
                continue
            elif (P1.x_exp[i, current_qubit] == 1 and P1.z_exp[i, current_qubit] == 0) or (P1.x_exp[i, current_qubit] == 0 and P1.z_exp[i, current_qubit] == 1): # X, Z
                if i not in flatten_list(cc_bands):
                    current_qubit_pauli_check = False
                    break
                else:
                    for b2 in cc_bands:
                        if i in b2:
                            if sum([P1.x_exp[j, current_qubit] for j in b2]) - sum([P1.z_exp[j, current_qubit] for j in b2]) == 0:
                                continue
                            else:
                                current_qubit_pauli_check = False
                                break
    return(current_qubit_pauli_check)

def remove_Ys(P1,C,current_qubit,q_print = False):
    cq = current_qubit
    sym_paulis_candidates = [i for i in range(P1.n_paulis()) if (P1.x_exp[i,cq] == 1 and P1.z_exp[i,cq] == 0) or (P1.x_exp[i,cq] == 0 and P1.z_exp[i,cq] == 1)]
    if q_print:
        print('sym_paulis_candidates',sym_paulis_candidates)
    other_paulis = [i for i in range(P1.n_paulis()) if i not in sym_paulis_candidates]
    if q_print:
        print('other_paulis',other_paulis)
    P2 = P1.copy()
    C2 = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    for g in C.gates:
        C2.add_gate(g)
    h_sym = False
    h_sym_impossible = False
    no_forcings = False
    no_qubits = False
    usable_qubits = [i for i in range(cq+1,P1.n_qudits())]
    gate_options = [[0,1,2,3] for i in range(cq+1,P1.n_qudits())]
    #print(gate_options)
    while not no_forcings and len(usable_qubits) > 0 and not h_sym and not h_sym_impossible:
        list_n_I = [number_of_I(P2,Pauli_index,usable_qubits) for Pauli_index in range(P2.n_paulis())]
        if q_print:
            print('list_n_I',list_n_I)
        forcing_candidates_paulis = [i for i in range(P2.n_paulis()) if list_n_I[i] == len(usable_qubits) - 1 and i not in sym_paulis_candidates]
        if q_print:
            print('forcing_candidates_paulis',forcing_candidates_paulis)
        forcing_candidates_qubits = []
        for i in forcing_candidates_paulis:
            qubit_index = min([j for j in usable_qubits if (P2.x_exp[i,j]+P2.z_exp[i,j]) != 0])
            if qubit_index not in forcing_candidates_qubits:
                forcing_candidates_qubits.append(qubit_index)
        if q_print:
            print('forcing_candidates_qubits',forcing_candidates_qubits)
        if len(forcing_candidates_qubits) == 0:
            no_forcings = True

        no_forcings = True
        for qi in forcing_candidates_qubits:
            pauli_indexes = [i for i in forcing_candidates_paulis if (P2.x_exp[i,qi]+P2.z_exp[i,qi]) != 0]
            if q_print:
                print('pauli_indexes',pauli_indexes)
            first_qubit_x = P2.x_exp[[j for j in pauli_indexes],cq]
            if q_print:
                print('first_qubit_x',first_qubit_x)
            forcing_pauli_types = set([])
            for i in pauli_indexes:
                if P2.x_exp[i,qi] == 0 and P2.z_exp[i,qi] == 0:
                    forcing_pauli_types.add(0)
                elif P2.x_exp[i,qi] == 1 and P2.z_exp[i,qi] == 0:
                    forcing_pauli_types.add(1)
                elif P2.x_exp[i,qi] == 0 and P2.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(2)
                elif P2.x_exp[i,qi] == 1 and P2.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(3)
            if q_print:
                print('forcing_pauli_types',forcing_pauli_types)

            if len(pauli_indexes) > 1 and len(forcing_pauli_types) > 1:
                if not any(first_qubit_x):
                    #print(gate_options[qi-1])
                    gate_options[qi-(1+cq)].remove(1)
                    gate_options[qi-(1+cq)].remove(2)
                    gate_options[qi-(1+cq)].remove(3)
                    usable_qubits.remove(qi)
                    #print('identity',qi)
                    #print(P2)
                    #print()
                    # identity
                elif not any((first_qubit_x + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                    gate_options[qi-(1+cq)].remove(0)
                    gate_options[qi-(1+cq)].remove(2)
                    gate_options[qi-(1+cq)].remove(3)
                    g = add_r2(P1,cq,qi)
                    P2 = g.act(P2)
                    for gg in g.gates:
                        C2.add_gate(gg)
                    usable_qubits.remove(qi)
                    #print('Add r2',qi)
                    #print(P2)
                    #print()
                elif not any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi])%2):
                    gate_options[qi-(1+cq)].remove(0)
                    gate_options[qi-(1+cq)].remove(1)
                    gate_options[qi-(1+cq)].remove(3)
                    g = add_s2(P1,cq,qi)
                    P2 = g.act(P2)
                    for gg in g.gates:
                        C2.add_gate(gg)
                    usable_qubits.remove(qi)
                    #print('Add s2',qi)
                    #print(P2)
                    #print()
                elif not any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi] + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                    #print(gate_options[qi-1])
                    gate_options[qi-(1+cq)].remove(0)
                    gate_options[qi-(1+cq)].remove(1)
                    gate_options[qi-(1+cq)].remove(2)
                    g = add_r2s2(P1,cq,qi)
                    P2 = g.act(P2)
                    for gg in g.gates:
                        C2.add_gate(gg)
                    usable_qubits.remove(qi)
                    #print('Add r2s2',qi)
                    #print(P2)
                    #print()
                else:
                    gate_options[qi-(1+cq)].remove(0)
                    gate_options[qi-(1+cq)].remove(1)
                    gate_options[qi-(1+cq)].remove(2)
                    gate_options[qi-(1+cq)].remove(3)
                    h_sym_impossible = True
                    #print('Not Possible')
                    usable_qubits.remove(qi)
                no_forcings = False
                break
            else:
                continue
        first_qubit_x = P2.x_exp[other_paulis,cq]
        if q_print:
            print('usable_qubits',usable_qubits)
            print('gate_options',gate_options)
        if not any(first_qubit_x%2):
            h_sym = True
        #if q_print:
        #print(P2)
    
    if q_print:
        print()
        print('h_sym',h_sym)
        print('h_sym_impossible',h_sym_impossible)
    if not h_sym and not h_sym_impossible:
        addition_options_lens = []
        for l in gate_options:
            addition_options_lens.append(len(l))
        max_options = np.prod(addition_options_lens)
        for i in range(max_options):
            addition_options = int_to_bases(i,dims=addition_options_lens)
            first_qubit_x = P2.x_exp[[j for j in range(P1.n_paulis()) if j not in sym_paulis_candidates],cq]
            if q_print:
                print('addition_options',addition_options)
                print('first_qubit_x',first_qubit_x)
            for k in range(len(addition_options)):
                if addition_options[k] == 0:
                    pass
                elif addition_options[k] == 1:
                    first_qubit_x += P2.x_exp[other_paulis,k+1+cq]
                elif addition_options[k] == 2:
                    first_qubit_x += P2.z_exp[other_paulis,k+1+cq]
                elif addition_options[k] == 3:
                    first_qubit_x += P2.x_exp[other_paulis,k+1+cq] + P2.z_exp[other_paulis,k+1+cq]
            #print(first_qubit_x%2)
            if q_print:
                print('first_qubit_x',first_qubit_x)
            if not any(first_qubit_x%2):
                #print('found solution')
                #print(addition_options)
                for j,k in enumerate(addition_options):
                    if addition_options_lens[j] > 1:
                        if k == 0:
                            #print(j,k)
                            #print('Identity',qi)
                            #print(P2)
                            #print()
                            pass
                        elif k == 1:
                            #print(j,k)
                            g = add_r2(P1,cq,j+1+cq)
                            P2 = g.act(P2)
                            for gg in g.gates:
                                C2.add_gate(gg)
                            #print('Add r2',qi)
                            #print(P2)
                            #print()
                        elif k == 2:
                            #print(j,k)
                            g = add_s2(P1,cq,j+1+cq)
                            P2 = g.act(P2)
                            for gg in g.gates:
                                C2.add_gate(gg)
                            #print('Add s2',qi)
                            #print(P2)
                            #print()
                        elif k == 3:
                            #print(j,k)
                            g = add_r2s2(P1,cq,j+1+cq)
                            P2 = g.act(P2) 
                            for gg in g.gates:
                                C2.add_gate(gg)
                            #print('Add r2s2',qi)
                            #print(P2)
                            #print()
                break
    return(P2,C2)

In [48]:
def PauliSum_Subspace(P,paulis,qubits):
    pauli_strings = P.pauli_strings
    weights = P.weights
    dimensions = P.dimensions
    phases = P.phases
    n_qudits = P.n_qudits()
    n_paulis = P.n_paulis()
    
    dimensions_new = np.array(dimensions)[qubits]
    pauli_strings_new = [PauliString(pauli_strings[i].x_exp[qubits],pauli_strings[i].z_exp[qubits],dimensions=dimensions_new) for i in paulis]
    weights_new = weights[paulis]
    phases_new = phases[paulis]
    P_new = PauliSum(pauli_strings_new, weights=weights_new ,dimensions=dimensions_new, phases=phases_new,standardise=False)
    return(P_new)

def add_uneven_phase(P1,q1,q2):
    C = Circuit(dimensions=[2 for i in range(P1.n_qudits())],gates = [CX(q2,q1,2),S(q1,2),CX(q2,q1,2),CX(q1,q2,2),S(q2,2),CX(q1,q2,2)])
    return(C)

In [ ]:
def Find_Hadamard_Symmetries(P,q_print=False):
    P1 = P.copy()
    C = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    # construct a list of all possible PS combinations that have the same absolute value coefficient 
    cc = P1.weights
    cc_abs = np.abs(cc)
    cc_bands = group_indices(cc_abs)
    cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]
    current_qubit = 0
    for ib,b in enumerate(cc_bands):
        for ic,pi in enumerate(b):
            for jc,pj in enumerate(b[ic+1:]):
                P_temp = PauliSum_Subspace(P1,[pi,pj],[i for i in range(current_qubit,P1.n_qudits())])
                if not P_temp.is_commuting():
                    P1,C,current_qubit = Find_Hadamard_Symmetries_iter_(P1,C,pi,pj,current_qubit,cc_bands)
                    if q_print:
                        print(pi,pj)
                        print(P1)
                        print()
    return(P1,C)

def Find_Hadamard_Symmetries_iter_(P1,C,pi,pj,current_qubit,cc_bands,q_print=False):
    C1 = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    #for g in C.gates:
    #    C1.add_gate(g)
    
    P2 = P1.copy()
    C1, P2, px, pz = prepare_sym_candidates(P2,pi,pj,C1,current_qubit)
    if q_print:
        print('Original')
        print(P2)
        print()
        print('Circuit')
        print(C1.act(P1.copy()))
    

    # cancel for pauli with x
    P2, C1 = cancel_pauli(P2, current_qubit, px, C1, P2.n_qudits())
    # cancel for pauli with z
    g = H(current_qubit, P2.dimensions[current_qubit])
    C1.add_gate(g)
    P2 = g.act(P2)
    P2, C1 = cancel_pauli(P2, current_qubit, pz, C1, P2.n_qudits())  
    g = H(current_qubit, P2.dimensions[current_qubit])
    C1.add_gate(g)
    P2 = g.act(P2)
    if q_print:
        print('Make X111... and Z111...')
        print(P2)
        #print(C1.act(P1.copy()))


    current_qubit_pauli_check = check_current_paulis(P2,pi,pj,current_qubit,cc_bands)
    #print('current_qubit_pauli_check',current_qubit_pauli_check)
    if not current_qubit_pauli_check:
        # not a candidate for Hadamard symmetry, return just the original P1 and C
        return(P1,C,current_qubit)

    for i in range(current_qubit+1,P1.n_qudits()):
        #print(i)
        C1, pivots = symplectic_reduction_iter_qudit_(P1.copy(), C1, [], i)
        #print(C1.act(P1.copy()))

    P2 = C1.act(P1.copy())
    if q_print:
        print(P2)

    # cancel the Y's in the first qubit
    #print(P2)
    P2,C1 = remove_Ys(P2,C1,current_qubit,q_print=q_print)
    if q_print:    
        print(P2)

    # check if all Y's in the first qubit have been cancelled
    sym_paulis_candidates = [i for i in range(P2.n_paulis()) if (P2.x_exp[i,current_qubit] == 1 and P2.z_exp[i,current_qubit] == 0) 
    or (P2.x_exp[i,current_qubit] == 0 and P2.z_exp[i,current_qubit] == 1)]
    other_paulis = [i for i in range(P2.n_paulis()) if i not in sym_paulis_candidates]
    first_qubit_x = P2.x_exp[other_paulis,current_qubit]
    if any(first_qubit_x%2):
        #print('No Hadamard Symmetry')
        return(P1,C,current_qubit)
    
    # deal with phases
    # 
    cq = current_qubit
    
    sym_pairs = set([])
    for i,spc_i in enumerate(sym_paulis_candidates):
        for j,spc_j in enumerate(sym_paulis_candidates[i+1:]):
            if abs(P2.weights[spc_i]) == abs(P2.weights[spc_j]) and np.array_equal(P2.x_exp[spc_i,current_qubit+1:],P2.x_exp[spc_j,current_qubit+1:]) and np.array_equal(P2.z_exp[spc_i,current_qubit+1:],P2.z_exp[spc_j,current_qubit+1:]):
                sym_pairs.add((spc_i,spc_j))
    #print(sym_pairs)
    
    pairs_parity = []
    for pairs in sym_pairs:
        if abs(P2.weights[pairs[0]] * xi(P2.phases[pairs[0]],2) - P2.weights[pairs[1]] * xi(P2.phases[pairs[1]],2)) < 10**-10:
            pairs_parity.append(0)
        else:
            pairs_parity.append(1)
    #print(pairs_parity)
    max_options = 8**(P.n_qudits() - current_qubit - 1)
    for i in range(max_options):
        phase_options = int_to_bases(i,dims=[8]*(P.n_qudits() - current_qubit - 1))
        #print(phase_options)
        pairs_parity_copy = pairs_parity.copy()
        for k in range(len(phase_options)):
            #print(k)
            if phase_options[k] == 0: # SSSS
                pass
            elif phase_options[k] == 1: # DDSS
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pass
                #
            elif phase_options[k] == 2: # DSDS
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                #
            elif phase_options[k] == 3: # DSSD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                #
            elif phase_options[k] == 4: # SDDS
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                #
            elif phase_options[k] == 5: # SDSD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                #
            elif phase_options[k] == 6: # SSDD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                #
            elif phase_options[k] == 7: # DDDD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
        if q_print:
            print(pairs_parity_copy)
        if not any(pairs_parity_copy):
            for j,k in enumerate(phase_options):
                if k == 0:
                    #print(j,k)
                    #print('Identity',qi)
                    #print(P2)
                    #print()
                    pass
                elif k == 1:
                    #print(j,k)
                    g = add_phase(P1,cq,j+1+cq,'DDSS')
                    P2 = g.act(P2)
                    for gg in g.gates:
                        C1.add_gate(gg)
                    #print(P2)
                    #print()
                elif k == 2:
                    #print(j,k)
                    g = add_phase(P1,cq,j+1+cq,'DSDS')
                    P2 = g.act(P2)
                    for gg in g.gates:
                        C1.add_gate(gg)
                    #print(P2)
                    #print()
                elif k == 3:
                    g = add_phase(P1,cq,j+1+cq,'DSSD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 4:
                    g = add_phase(P1,cq,j+1+cq,'SDDS')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 5:
                    g = add_phase(P1,cq,j+1+cq,'SDSD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 6:
                    g = add_phase(P1,cq,j+1+cq,'SSDD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 7:
                    g = add_phase(P1,cq,j+1+cq,'DDDD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
            break
            
    # ToDo: final check for Hadamard symmetry
    # only x and z in the first qubit if both have the same coefficient
    # no problematic phases
    # after the x and z, all paulis are the same


    C2 = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    for g in C.gates:
        C2.add_gate(g)
    for g in C1.gates:
        C2.add_gate(g)

    return(P2,C2,current_qubit+1)
    

In [64]:
P,C = Hadamard_Symmetric_PauliSum(25,8,4)
print(P)

[ 0.46693172 -1.20041766]
[0.94856989]
[-2.50444326  0.87865108  0.15973994]
[-0.18646676]
[4, 18, 11, 1, 21, 23, 15] [5, 17, 10, 0, 20, 24, 16]
(-2.504443260343078+0j) |x0z1 x1z1 x0z0 x0z1 x0z1 x0z0 x0z0 x1z1 | 0 
(-2.504443260343078+0j) |x0z1 x1z1 x1z0 x0z1 x0z1 x0z0 x0z1 x0z1 | 0 
(1.0537748759681365+0j) |x1z0 x1z0 x0z0 x1z1 x1z1 x0z1 x0z1 x0z0 | 0 
(0.8453444169091787-0j) |x1z1 x1z0 x0z0 x1z1 x0z1 x1z1 x1z0 x1z1 | 0 
(0.4669317191151887+0j) |x1z0 x1z0 x0z1 x1z1 x0z0 x0z1 x1z0 x1z0 | 0 
(-0.4669317191151887+0j)|x1z0 x0z1 x1z0 x1z1 x0z1 x0z0 x1z0 x0z1 | 0 
(2.1770858571753755+0j) |x0z1 x1z1 x1z1 x0z1 x0z1 x0z0 x0z0 x1z1 | 0 
(-1.4096990085158183+0j)|x0z1 x0z0 x1z1 x0z1 x0z1 x1z1 x0z1 x0z1 | 0 
(-1.0564903258616098+0j)|x1z0 x0z1 x1z1 x1z0 x0z0 x1z1 x1z0 x0z0 | 0 
(-0.4521837172758721+0j)|x0z1 x0z1 x1z0 x0z1 x0z0 x0z0 x0z1 x0z0 | 0 
(-0.9485698930941907+0j)|x1z1 x1z1 x0z1 x1z0 x0z1 x0z0 x1z0 x1z0 | 0 
(0.9485698930941907+0j) |x1z1 x0z1 x0z1 x0z1 x0z0 x1z0 x1z1 x1z0 | 0 
(1.298308030291

In [67]:
P1,C = Find_Hadamard_Symmetries(P)
print(P1)

(-2.504443260343078+0j) |x0z1 x0z0 x0z0 x0z0 x0z0 x0z1 x0z0 x0z0 | 0 
(-2.504443260343078+0j) |x1z0 x0z0 x0z0 x0z0 x0z0 x0z1 x0z0 x0z0 | 0 
(1.0537748759681365+0j) |x0z0 x0z0 x0z0 x0z0 x0z1 x0z0 x0z0 x0z0 | 1 
(0.8453444169091787-0j) |x0z0 x0z0 x0z0 x0z0 x0z1 x1z1 x1z0 x0z0 | 0 
(0.4669317191151887+0j) |x0z0 x0z1 x0z0 x0z0 x1z0 x0z0 x0z0 x0z0 | 1 
(-0.4669317191151887+0j)|x0z0 x1z0 x0z0 x0z0 x1z0 x0z0 x0z0 x0z0 | 0 
(2.1770858571753755+0j) |x0z0 x0z0 x0z0 x0z0 x0z1 x0z0 x0z1 x0z0 | 0 
(-1.4096990085158183+0j)|x0z0 x0z0 x0z0 x0z0 x0z0 x1z0 x1z1 x1z0 | 1 
(-1.0564903258616098+0j)|x0z0 x0z0 x0z0 x0z0 x0z0 x1z1 x1z1 x0z0 | 1 
(-0.4521837172758721+0j)|x0z0 x0z0 x0z0 x0z0 x1z1 x1z1 x1z1 x0z0 | 1 
(-0.9485698930941907+0j)|x0z0 x0z0 x1z0 x0z0 x1z1 x1z0 x1z1 x0z1 | 1 
(0.9485698930941907+0j) |x0z0 x0z0 x0z1 x0z0 x1z1 x1z0 x1z1 x0z1 | 0 
(1.298308030291583-0j)  |x0z0 x0z0 x0z0 x0z0 x1z1 x1z0 x0z0 x0z0 | 1 
(0.4571946673692935+0j) |x0z0 x0z0 x0z0 x0z0 x1z1 x1z0 x0z1 x1z0 | 1 
(-0.4571946673692935

In [50]:
cc = P.weights
cc_abs = np.abs(cc)
cc_bands = group_indices(cc_abs)
cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]
C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
print(P)
print()
P1,C,current_qubit = Find_Hadamard_Symmetries_iter_(P,C,1,2,0,cc_bands)
#print(current_qubit)
P1,C,current_qubit = Find_Hadamard_Symmetries_iter_(P1,C,11,12,current_qubit,cc_bands,q_print=False)
#P1,C,current_qubit = Find_Hadamard_Symmetries_iter_(P1,C,23,24,current_qubit,cc_bands,q_print=True)
#P1,C = Find_Hadamard_Symmetries(P)
print(P1)
print(P1 == C.act(P))

(-1.595975519615152+0j)   |x1z0 x1z1 x0z0 x0z0 x1z0 x1z1 x0z0 x0z1 | 0 
(1.595975519615152-0j)    |x0z0 x1z1 x0z1 x0z1 x0z1 x1z1 x1z0 x1z1 | 0 
(1.595975519615152-0j)    |x1z0 x0z1 x1z1 x1z0 x1z1 x0z1 x0z1 x1z0 | 0 
(-0.7028090543370635+0j)  |x0z1 x1z0 x0z0 x0z1 x0z0 x1z1 x0z0 x1z1 | 0 
(-0.7063063566636919+0j)  |x1z1 x1z1 x0z0 x1z1 x1z0 x0z0 x1z0 x1z0 | 0 
(0.6969201489230441-0j)   |x0z0 x1z1 x0z0 x1z1 x1z0 x0z1 x1z1 x1z1 | 0 
(0.6969201489230441-0j)   |x1z0 x0z1 x1z0 x0z0 x0z0 x1z1 x0z0 x1z0 | 0 
(2.3484203129711583+0j)   |x0z1 x0z0 x0z1 x1z1 x0z0 x1z0 x1z1 x0z0 | 0 
(-0.4548959358007366+0j)  |x0z1 x1z1 x1z0 x1z0 x0z1 x0z1 x1z0 x0z1 | 0 
(-0.8987870528114668+0j)  |x0z0 x1z1 x1z0 x1z0 x0z1 x0z1 x0z0 x0z1 | 0 
(1.6972646567523169+0j)   |x0z0 x1z1 x1z0 x1z1 x1z0 x0z1 x0z0 x0z1 | 0 
(0.7250958741620873-0j)   |x1z1 x0z0 x1z0 x0z1 x0z1 x1z1 x1z0 x1z0 | 0 
(-0.7250958741620873+0j)  |x1z0 x0z0 x1z0 x1z1 x0z0 x1z1 x0z0 x1z1 | 0 
(-1.6098063363418227+0j)  |x1z0 x1z0 x1z0 x0z0 x1z1 x1z0 x1z0 x0

In [18]:
def find_circuit_simultaneous(start_paulis,goal_paulis,iterations,compare_phases = True):
    n_qudits = start_paulis[0].n_qudits()
    SUMs = [CX(i, j, 2) for i in range(n_qudits) for j in range(n_qudits) if i != j]
    Ss = [S(i, 2) for i in range(n_qudits)]
    Hs = [H(i, 2) for i in range(n_qudits)]
    all_gates = SUMs + Ss + Hs

    goal_circuits = []
    circuits = [[Circuit(dimensions=[2 for i in range(n_qudits)])] for _ in range(len(start_paulis))]
    intermediate_paulis = [[start_paulis[i]] for i in range(len(start_paulis))]
    
    for i in range(iterations):
        print(i)
        intermediate_paulis_new = [[] for _ in range(len(start_paulis))]
        circuits_new = [[] for _ in range(len(start_paulis))]
        for g in all_gates:
            for ip in range(len(intermediate_paulis[0])):
                current_paulis = []
                for ib,b in enumerate(intermediate_paulis):
                    p = b[ip]
                    P_temp = g.act(p)
                    if not compare_phases:
                        P_temp.phases = [0]
                    C_temp = Circuit(dimensions=[2 for i in range(n_qudits)])
                    for g2 in circuits[ib][ip].gates:
                        C_temp.add_gate(g2)
                    C_temp.add_gate(g)
                    intermediate_paulis_new[ib].append(P_temp)
                    circuits_new[ib].append(C_temp)
                    current_paulis.append(P_temp)
                if all([current_paulis[j] in goal_paulis for j in range(len(current_paulis))]):
                    goal_circuits.append(circuits[ib][ip])
                    return(C_temp)
                    #print("Found goal circuit")
        intermediate_paulis = intermediate_paulis_new.copy()
        circuits = circuits_new
    return(None)

In [139]:
C = add_r2s2(P,0,1)
symplectic_effect(C)

omega**(r2*s1 + s1*s2)*(X**(r1 + r2 + s2)*Z**(r2 + s1 + s2))x(X**(r1 + r2 + s1)*Z**(r2 + s2))x(X**r3*Z**s3)x(X**r4*Z**s4)x(X**r5*Z**s5)x(X**r6*Z**s6)x(X**r7*Z**s7)x(X**r8*Z**s8)

()

In [ ]:
start_pauli = PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0])
goal_pauli = PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0])
iterations = 8
goal_circuits = find_circuit(start_pauli,goal_pauli,iterations,compare_phases=False)
for i,c in enumerate(goal_circuits):
    p = PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0])
    for g in c.gates:
        p = g.act(p)
    print(p)
    symplectic_effect(c)

SORTING
SORTING
0 1 0
1 5 0
2 19 0
3 55 1
4 104 5
5 119 11
6 120 17
7 120 23
SORTING
(1+0j)|x1z0 x0z1 | 1 
(1+0j)|x0z1 x0z1 | 0 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**r2)

SORTING
(1+0j)|x1z0 x0z1 | 1 
(1+0j)|x0z1 x0z1 | 0 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**r2)

SORTING
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 



...

omega**(r1*r2 + r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + s1 + s2)*Z**r2)

SORTING
(1+0j)|x1z0 x0z1 | 1 
(1+0j)|x0z1 x0z1 | 0 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**r2)

SORTING
(1+0j)|x1z0 x0z1 | 1 
(1+0j)|x0z1 x0z1 | 0 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**r2)

In [145]:
p = PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0])
C = add_r2s2(p,0,1)
p = C.act(p)
print(p)

SORTING
(1+0j)|x0z1 x1z0 | 0 
(1+0j)|x1z0 x1z0 | 0 



In [150]:
for i,c in enumerate(goal_circuits):
    p = PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0])
    for g in c.gates:
        p = g.act(p)
    print(c)
    print(p)
    symplectic_effect(c)

SORTING
SUM [1, 0]
H [1]
SUM [0, 1]
S [1]

(1+0j)|x1z0 x1z0 | 0 
(1+0j)|x0z1 x1z0 | 1 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**(r1 + s1 + s2))

SORTING
SUM [1, 0]
H [1]
SUM [0, 1]
S [1]

(1+0j)|x1z0 x1z0 | 0 
(1+0j)|x0z1 x1z0 | 1 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**(r1 + s1 + s2))

SORTING
SUM [0, 1]
H [0]
SUM [0, 1]
S [1]
H [0]

(1+0j)|x1z0 x1z0 | 0 
(1+0j)|x0z1 x1z0 | 1 



...

omega**(r1*r2 + r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + s1 + s2)*Z**(r1 + r2 + s1 + s2))

SORTING
SUM [0, 1]
SUM [1, 0]
H [0]
SUM [0, 1]
S [1]
SUM [1, 0]

(1+0j)|x1z0 x1z0 | 0 
(1+0j)|x0z1 x1z0 | 1 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**(r1 + s1 + s2))

SORTING
SUM [1, 0]
SUM [0, 1]
S [0]
H [1]
S [1]
SUM [0, 1]

(1+0j)|x1z0 x1z0 | 1 
(1+0j)|x0z1 x1z0 | 1 



omega**(r1*s1 + r1*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**(r1 + s1 + s2))

In [69]:
start_pauli = [PauliSum(['x0z0 x1z0', 'x0z0 x1z0'], dimensions=[2, 2], phases=[1,0]),
               PauliSum(['x0z0 x1z0', 'x0z0 x1z0'], dimensions=[2, 2], phases=[0,1])]
goal_pauli = [PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z0 x1z0', 'x0z0 x1z0'], dimensions=[2, 2], phases=[1,1]),
               PauliSum(['x0z0 x0z1', 'x0z0 x0z1'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z0 x0z1', 'x0z0 x0z1'], dimensions=[2, 2], phases=[1,1]),
               PauliSum(['x0z0 x1z1', 'x0z0 x1z1'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z0 x1z1', 'x0z0 x1z1'], dimensions=[2, 2], phases=[1,1]),
               PauliSum(['x0z0 x1z0', 'x0z0 x1z0'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z0 x1z0', 'x0z0 x1z0'], dimensions=[2, 2], phases=[1,1]),
               PauliSum(['x0z0 x0z1', 'x0z0 x0z1'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z0 x0z1', 'x0z0 x0z1'], dimensions=[2, 2], phases=[1,1]),
               PauliSum(['x0z0 x1z1', 'x0z0 x1z1'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z0 x1z1', 'x0z0 x1z1'], dimensions=[2, 2], phases=[1,1])]

iterations = 8
goal_circuit = find_circuit_simultaneous(start_pauli,goal_pauli,iterations,compare_phases = True)
print(goal_circuit)
symplectic_effect(goal_circuit)

SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING


KeyboardInterrupt: 

In [67]:
C = goal_circuit
p = PauliSum(['x0z0 x1z1', 'x0z0 x1z1'], dimensions=[2, 2], phases=[0,1])
print(C.act(p))

SORTING
(1+0j)|x1z1 x1z0 | 1 
(1+0j)|x1z1 x1z0 | 0 



In [62]:
start_pauli = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z1 x0z0', 'x1z0 x0z0'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z1 x1z0', 'x1z0 x1z0'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z1 x0z1', 'x1z0 x0z1'], dimensions=[2, 2], phases=[0,0]),
               PauliSum(['x0z1 x1z1', 'x1z0 x1z1'], dimensions=[2, 2], phases=[0,0])]
goal_pauli = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,1]),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,1]),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,1]),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,1]),
              PauliSum(['x0z1 x0z0', 'x1z0 x0z0'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x0z1 x1z0', 'x1z0 x1z0'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x0z1 x0z1', 'x1z0 x0z1'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x0z1 x1z1', 'x1z0 x1z1'], dimensions=[2, 2], phases=[1,0]),
              PauliSum(['x0z1 x0z0', 'x1z0 x0z0'], dimensions=[2, 2], phases=[0,1]),
              PauliSum(['x0z1 x1z0', 'x1z0 x1z0'], dimensions=[2, 2], phases=[0,1]),
              PauliSum(['x0z1 x0z1', 'x1z0 x0z1'], dimensions=[2, 2], phases=[0,1]),
              PauliSum(['x0z1 x1z1', 'x1z0 x1z1'], dimensions=[2, 2], phases=[0,1])]

iterations = 8
goal_circuit = find_circuit_simultaneous(start_pauli,goal_pauli,iterations,compare_phases = True)
print(goal_circuit)
symplectic_effect(goal_circuit)


SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
SORTING
0
1
2
3
4
5
SUM [1, 0]
S [0]
SUM [1, 0]
SUM [0, 1]
S [1]
SUM [0, 1]



omega**r2*(X**r1*Z**s1)x(X**r2*Z**s2)

()

In [172]:
p_test = PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False)
goal_circuit = add_r2s2(p_test,0,1)
symplectic_effect(goal_circuit)
print(goal_circuit)
print(goal_circuit.act(p_test))

omega**(r2*s1 + s1*s2)*(X**(r1 + r2 + s2)*Z**(r2 + s1 + s2))x(X**(r1 + r2 + s1)*Z**(r2 + s2))

S [1]
SUM [0, 1]
H [1]
SUM [1, 0]
S [1]
H [1]

(1+0j)|x1z0 x1z0 | 0 
(1+0j)|x0z1 x1z0 | 0 



In [175]:
def find_circuit_V2(start_pauli,goal_pauli,iterations,compare_phases = False,employ_reached_paulis = True):
    n_qudits = start_pauli.n_qudits()
    SUMs = [CX(i, j, 2) for i in range(n_qudits) for j in range(n_qudits) if i != j]
    Ss = [S(i, 2) for i in range(n_qudits)]
    Hs = [H(i, 2) for i in range(n_qudits)]
    all_gates = SUMs + Ss + Hs

    intermediate_paulis = [start_pauli.copy()]
    intermediate_paulis_new = [start_pauli.copy()]
    reached_paulis = [start_pauli.copy()]
    intermediate_circuits = [Circuit(dimensions=[2 for i in range(n_qudits)])]
    intermediate_circuits_new = [Circuit(dimensions=[2 for i in range(n_qudits)])]
    
    goal_circuits = []

    for i in range(iterations):
        print(i,len(intermediate_paulis_new),len(goal_circuits))
        intermediate_paulis = intermediate_paulis_new.copy()
        intermediate_circuits = intermediate_circuits_new.copy()
        intermediate_paulis_new = []
        intermediate_circuits_new = []
        for ip,p in enumerate(intermediate_paulis):
            for g in all_gates:
                if len(intermediate_circuits[ip].gates) > 0:
                    if g == intermediate_circuits[ip].gates[-1]:
                        continue
                # create new pauli
                P_temp = g.act(p)
                if not compare_phases:  
                    P_temp.phases = [0]
                # append circuit 
                C_temp = Circuit(dimensions=[2 for i in range(n_qudits)])
                for g2 in intermediate_circuits[ip].gates:
                    C_temp.add_gate(g2)
                C_temp.add_gate(g)
                if P_temp == goal_pauli:
                    if C_temp not in goal_circuits:
                        goal_circuits.append(C_temp)
                else:
                    if P_temp in reached_paulis and employ_reached_paulis:
                        pass
                    else:
                        reached_paulis.append(P_temp)
                        intermediate_paulis_new.append(P_temp)
                        intermediate_circuits_new.append(C_temp)

    return(goal_circuits)



In [ ]:
start_pauli = PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False)
goal_paulis = PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False)
iterations = 8
goal_circuits = find_circuit_V2(start_pauli,goal_pauli,iterations,compare_phases=True,employ_reached_paulis=True)
for i,c in enumerate(goal_circuits):
    p = PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0])
    for g in c.gates:
        p = g.act(p)
    print(p)
    symplectic_effect(c)


0 1 0
1 4 0
2 14 0
3 40 1
4 78 4
5 122 5
6 120 6
7 77 6
SORTING
(1+0j)|x1z0 x0z1 | 1 
(1+0j)|x0z1 x0z1 | 0 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**r2)

SORTING
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 



omega**(s1*s2 + s2)*(X**(r1 + s2)*Z**(s1 + s2))x(X**(r1 + r2 + s1 + s2)*Z**s2)

SORTING
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 



...

omega**(r1*r2 + r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + s1 + s2)*Z**r2)

SORTING
(1+0j)|x1z0 x0z1 | 1 
(1+0j)|x0z1 x0z1 | 0 



omega**(r2*s1 + r2*s2)*(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + r2 + s1 + s2)*Z**r2)

SORTING
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 



omega**(s1*s2 + s2)*(X**(r1 + s2)*Z**(s1 + s2))x(X**(r1 + r2 + s1 + s2)*Z**s2)

In [8]:
start_paulis = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0],standardise=False)]

goal_paulis = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x0z1 x0z0', 'x1z0 x0z0'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x0z1 x1z0', 'x1z0 x1z0'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x0z1 x0z1', 'x1z0 x0z1'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x0z1 x1z1', 'x1z0 x1z1'], dimensions=[2, 2], phases=[1,0],standardise=False),
              PauliSum(['x0z1 x0z0', 'x1z0 x0z0'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x0z1 x1z0', 'x1z0 x1z0'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x0z1 x0z1', 'x1z0 x0z1'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x0z1 x1z1', 'x1z0 x1z1'], dimensions=[2, 2], phases=[0,1],standardise=False),
              PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
              PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[1,1],standardise=False),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[1,1],standardise=False),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[1,1],standardise=False),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[1,1],standardise=False),
              PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[1,1],standardise=False),
              PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[1,1],standardise=False),
              PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[1,1],standardise=False),
              PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[1,1],standardise=False)]


n_qudits = 2
SUMs = [CX(i, j, 2) for i in range(n_qudits) for j in range(n_qudits) if i != j]
Ss = [S(i, 2) for i in range(n_qudits)]
Hs = [H(i, 2) for i in range(n_qudits)]
all_gates = SUMs + Ss + Hs

iterations = 10
paulis_reached = start_paulis.copy()
reached_circuits = [Circuit(dimensions=[2 for i in range(n_qudits)])]
intermediate_circuits = [Circuit(dimensions=[2 for i in range(n_qudits)])]
intermediate_circuits_new = [Circuit(dimensions=[2 for i in range(n_qudits)])]

goal_circuits = []

for i in range(iterations):
    print()
    print(i,len(intermediate_circuits_new),len(goal_circuits),len(paulis_reached))
    intermediate_circuits = intermediate_circuits_new.copy()
    intermediate_circuits_new = []
    for ic,c in enumerate(intermediate_circuits):
        if ic%10 == 0:
            print(ic)
        #print('c')
        #print(c)
        for g in all_gates:
            #print('g')
            #print(g.name)
            if len(c.gates) > 0:
                if g == c.gates[-1]:
                    continue
            C_temp = Circuit(dimensions=[2 for i in range(n_qudits)])
            for g2 in c.gates:
                C_temp.add_gate(g2)
            C_temp.add_gate(g)
            #print('C_temp')
            #print(C_temp)
            for ip,p in enumerate(start_paulis):
                P_temp = C_temp.act(p)
                #print('P_temp')
                #print(P_temp)
                if P_temp in goal_paulis:
                    names_bool = np.array([all([C_temp.gates[ig].name == g1.name for ig,g1 in enumerate(c2.gates)]) for c2 in goal_circuits])
                    indexes_bool = np.array([C_temp.indexes == c2.indexes  for c2 in goal_circuits])
                    circ_bool = names_bool * indexes_bool
                    if not any(circ_bool):
                        if P_temp != p and P_temp != Ss[1].act(p) and P_temp != Hs[0].act(p) and P_temp != Hs[1].act(p):
                            goal_circuits.append(C_temp)
                        else:
                            if P_temp not in paulis_reached:
                                intermediate_circuits_new.append(C_temp)
                                paulis_reached.append(P_temp)

                else:
                    if P_temp not in paulis_reached:
                        intermediate_circuits_new.append(C_temp)
                        paulis_reached.append(P_temp)

     
        


0 1 0 4
0

1 17 0 21
0
10

2 50 1 71
0
10
20
30
40

3 110 6 181
0
10
20
30
40
50
60
70
80
90
100

4 140 16 321
0
10
20
30
40
50
60
70
80
90
100
110
120
130

5 112 34 433
0
10
20
30
40
50
60
70
80
90
100
110

6 26 56 459
0
10
20

7 2 60 461
0

8 0 61 461

9 0 61 461


In [9]:
for i,c in enumerate(goal_circuits):
    print(c)
    symplectic_effect(c,phases=False)
    for ip,p in enumerate(start_paulis):
        P_temp = c.act(p)
        if ip == 0:
            ini_s = 'I'
        elif ip == 1:
            ini_s = 'X'
        elif ip == 2:
            ini_s = 'Z'
        elif ip == 3:
            ini_s = 'Y'

        if P_temp.x_exp[0,1] == 0:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','I')
            else:
                print(ini_s,' -> ','Z')
        elif P_temp.x_exp[0,1] == 1:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','X')
            else:
                print(ini_s,' -> ','Y')

        phases = P_temp.phases
        if phases[0] == phases[1]:
            print('Same Phase')
        else:
            print('Different Phase')
    print()
    print()
    print()

H [1]
S [1]



(X**r1*Z**s1)x(X**s2*Z**(r2 + s2))

I  ->  I
Same Phase
X  ->  Z
Same Phase
Z  ->  Y
Same Phase
Y  ->  X
Same Phase



SUM [0, 1]
H [0]
SUM [0, 1]



(X**(s1 + s2)*Z**(r1 + s2))x(X**(r1 + r2 + s1 + s2)*Z**s2)

I  ->  X
Same Phase
X  ->  I
Same Phase
Z  ->  Z
Different Phase
Y  ->  Y
Different Phase



SUM [0, 1]
H [1]
SUM [1, 0]



...

I  ->  I
Different Phase
X  ->  X
Different Phase
Z  ->  Y
Same Phase
Y  ->  Z
Same Phase



SUM [1, 0]
S [0]
H [1]
SUM [0, 1]
S [1]
H [1]
S [0]



(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + s1 + s2)*Z**(r1 + r2 + s1 + s2))

I  ->  Y
Same Phase
X  ->  X
Different Phase
Z  ->  I
Same Phase
Y  ->  Z
Different Phase



SUM [0, 1]
SUM [1, 0]
S [0]
H [0]
H [1]
S [1]
SUM [0, 1]
S [0]



(X**(r2 + s1 + s2)*Z**(r1 + r2 + s2))x(X**(r2 + s2)*Z**(r1 + r2 + s1))

I  ->  Z
Different Phase
X  ->  X
Different Phase
Z  ->  Y
Different Phase
Y  ->  I
Different Phase





In [11]:
from itertools import product

# Example list
elements = [1, 2, 3]

# Loop through all permutations of length 1 to n
for r in range(1, len(elements) + 1):
    for combo in product(elements, repeat=r):
        print(combo)

(1,)
(2,)
(3,)
(1, 1)
(1, 2)
(1, 3)
(2, 1)
(2, 2)
(2, 3)
(3, 1)
(3, 2)
(3, 3)
(1, 1, 1)
(1, 1, 2)
(1, 1, 3)
(1, 2, 1)
(1, 2, 2)
(1, 2, 3)
(1, 3, 1)
(1, 3, 2)
(1, 3, 3)
(2, 1, 1)
(2, 1, 2)
(2, 1, 3)
(2, 2, 1)
(2, 2, 2)
(2, 2, 3)
(2, 3, 1)
(2, 3, 2)
(2, 3, 3)
(3, 1, 1)
(3, 1, 2)
(3, 1, 3)
(3, 2, 1)
(3, 2, 2)
(3, 2, 3)
(3, 3, 1)
(3, 3, 2)
(3, 3, 3)


In [14]:



Id_phase_gates = [Circuit(dimensions=[2,2], gates=[H(1,2)]),
                  Circuit(dimensions=[2,2], gates=[S(1,2)]),
                  Circuit(dimensions=[2,2], gates=[H(1,2),S(1,2)]),
                  Circuit(dimensions=[2,2], gates=[S(1,2),H(1,2)]),
                  Circuit(dimensions=[2,2], gates=[S(0,2),H(1,2),S(1,2),H(1,2),S(0,2)]),
                  Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(1,2),S(1,2),H(1,2),CX(0,1,2)]),
                  Circuit(dimensions=[2,2], gates=[CX(0,1,2),S(1,2),H(1,2),S(1,2),CX(0,1,2)]),
                  Circuit(dimensions=[2,2], gates=[CX(1,0,2),S(0,2),CX(1,0,2),H(1,2),CX(0,1,2),S(0,2)]),
                  Circuit(dimensions=[2,2], gates=[CX(0,1,2),S(1,2),CX(0,1,2),H(1,2),CX(0,1,2),S(0,2)]),
                  Circuit(dimensions=[2,2], gates=[CX(0,1,2),S(0,2),S(1,2),CX(0,1,2),H(0,2),CX(1,0,2)]),
                  Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(0,2),H(0,2),CX(0,1,2),S(0,2),CX(1,0,2)])]

options = [0,1,2,3,4,5,6,7]

elements = [i for i in range(len(Id_phase_gates))]
for r in range(1, len(elements) + 1):
    print(r)
    print()
    for combo in product(elements, repeat=r):
        #print(combo)
        c = Circuit(dimensions=[2,2])
        for j in combo:
            C_add = Id_phase_gates[j]
            for g2 in C_add.gates:
                c.add_gate(g2)
        #print(c)
        #symplectic_effect(c,phases=False)
        p_counter = 0
        p_ss = ''
        ini_ss = []
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> I'
                    #print(ini_s,' -> ','I')
                    pass
                else:
                    ini_s += '-> Z'
                    #print(ini_s,' -> ','Z')
                    pass
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> X'
                    #print(ini_s,' -> ','X')
                    pass
                else:
                    ini_s += '-> Y'
                    #print(ini_s,' -> ','Y')
                    pass
            ini_ss.append(ini_s)
            phases = P_temp.phases
            if phases[0] == phases[1]:
                #print('Same Phase')
                p_ss += 'S'
                p_counter += 1
            else:
                p_ss += 'D'
                #print('Different Phase')
                pass
        if p_ss == 'SSSS' and 0 in options:
            print('Option 0:')
            print(c)
            print()
            options.remove(0)
        elif p_ss == 'DDSS' and 1 in options:
            print('Option 1:')
            print(c)
            print()
            options.remove(1)
        elif p_ss == 'DSDS' and 2 in options:
            print('Option 2:')
            print(c)
            print()
            options.remove(2)
        elif p_ss == 'DSSD' and 3 in options:
            print('Option 3:')
            print(c)
            print()
            options.remove(3)
        elif p_ss == 'SDDS' and 4 in options:
            print('Option 4:')
            print(c)
            print()
            options.remove(4)
        elif p_ss == 'SDSD' and 5 in options:
            print('Option 5:')
            print(c)
            print()
            options.remove(5)
        elif p_ss == 'SSDD' and 6 in options:
            print('Option 6:')
            print(c)
            print()
            options.remove(6)
        elif p_ss == 'DDDD' and 7 in options:
            print('Option 7:')
            print(c)
            print()
            options.remove(7)

        if p_counter == 3 or p_counter == 1:
            print(combo)
            print(c)
            symplectic_effect(c,phases=False)
            for ini_s in ini_ss:
                print(ini_s)
            print()
            print()
            print()

        if len(options) == 0:
            break

        
            
'''
for i,c in enumerate(Id_phase_gates):
    print(c)
    symplectic_effect(c,phases=False)
    for ip,p in enumerate(start_paulis):
        P_temp = c.act(p)
        if ip == 0:
            ini_s = 'I'
        elif ip == 1:
            ini_s = 'X'
        elif ip == 2:
            ini_s = 'Z'
        elif ip == 3:
            ini_s = 'Y'

        if P_temp.x_exp[0,1] == 0:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','I')
            else:
                print(ini_s,' -> ','Z')
        elif P_temp.x_exp[0,1] == 1:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','X')
            else:
                print(ini_s,' -> ','Y')

        phases = P_temp.phases
        if phases[0] == phases[1]:
            print('Same Phase')
        else:
            print('Different Phase')
    print()
    print()
    print()
'''

1

Option 0:
H [1]


Option 1:
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


Option 5:
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]


Option 2:
SUM [0, 1]
S [1]
SUM [0, 1]
H [1]
SUM [0, 1]
S [0]


2

Option 6:
H [1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]


Option 3:
S [1]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


Option 4:
S [1]
H [1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]


Option 7:
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


3

4

5

6

7

8

9

10

11



"\nfor i,c in enumerate(Id_phase_gates):\n    print(c)\n    symplectic_effect(c,phases=False)\n    for ip,p in enumerate(start_paulis):\n        P_temp = c.act(p)\n        if ip == 0:\n            ini_s = 'I'\n        elif ip == 1:\n            ini_s = 'X'\n        elif ip == 2:\n            ini_s = 'Z'\n        elif ip == 3:\n            ini_s = 'Y'\n\n        if P_temp.x_exp[0,1] == 0:\n            if P_temp.z_exp[0,1] == 0:\n                print(ini_s,' -> ','I')\n            else:\n                print(ini_s,' -> ','Z')\n        elif P_temp.x_exp[0,1] == 1:\n            if P_temp.z_exp[0,1] == 0:\n                print(ini_s,' -> ','X')\n            else:\n                print(ini_s,' -> ','Y')\n\n        phases = P_temp.phases\n        if phases[0] == phases[1]:\n            print('Same Phase')\n        else:\n            print('Different Phase')\n    print()\n    print()\n    print()\n"

In [265]:
p = PauliSum(['x1z0 x1z0', 'x0z1 x1z0', 'x0z0 x1z1','x0z0 x1z0','x0z0 x0z1','x1z0 x0z1','x0z1 x0z1'], dimensions=[2, 2], phases=[1,0,0,0,0,0,0],standardise=False)
#c = Circuit(dimensions=[2,2],gates=[CX(0,1,2),S(0,2),S(1,2),CX(0,1,2),H(0,2),CX(1,0,2)])
c = Circuit(dimensions=[2,2],gates=[CX(0,1,2),S(1,2),H(1,2),S(1,2),CX(0,1,2)])
print(p)
print()
print(c.act(p))

(1+0j)|x1z0 x1z0 | 1 
(1+0j)|x0z1 x1z0 | 0 
(1+0j)|x0z0 x1z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x1z0 x0z1 | 0 
(1+0j)|x0z1 x0z1 | 0 


(1+0j)|x1z0 x1z0 | 1 
(1+0j)|x0z1 x1z0 | 1 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 1 
(1+0j)|x0z0 x1z1 | 0 
(1+0j)|x1z0 x1z1 | 0 
(1+0j)|x0z1 x1z1 | 0 



In [ ]:



def Find_Hadamard_SymmetriesV1(P):
    # Step 0
    P1 = P.copy()
    #print(P1)
    cc = P1.weights
    # Step 1, organize the coefficients into bands with the same absolute value
    cc_abs = np.abs(cc)
    cc_bands = group_indices(cc_abs)
    cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]

    print(cc_bands)
    print([cc[b[0]] for b in cc_bands])

    for ib,b in enumerate(cc_bands):
        for ic,pi in enumerate(b):
            for jc,pj in enumerate(b[ic+1:]):
                P1 = P.copy()
                if not P1.is_commuting(pauli_string_indexes=(pi,pj)):
                    print(f"Band {ib}, Pauli {pi} and {pj} anti-commute")
                    #print()
                    C, P1, px, pz = prepare_sym_candidates(P1,pi,pj)

                    #print('Prepared:',pi,pj)
                    #print(P1)
                    #print()
                    # cancel for pauli with x
                    P1, C = cancel_pauli(P1, 0, px, C, P1.n_qudits())
                    # cancel for pauli with z
                    g = H(0, P.dimensions[0])
                    C.add_gate(g)
                    P1 = g.act(P1)
                    P1, C = cancel_pauli(P1, 0, pz, C, P1.n_qudits())  
                    g = H(0, P.dimensions[0])
                    C.add_gate(g)
                    P1 = g.act(P1)
                    #print('Make X111... and Z111...')
                    #print(P1)
                    #print()

                    # check if all other paulis in the first qubit have either id, Y, or x and z but with the same coefficients
                    current_qubit_pauli_check = check_current_paulis(P1,pi,pj)
                    #print('current_qubit_pauli_check',current_qubit_pauli_check)
                    if not current_qubit_pauli_check:
                        continue
                    
                    #print(pi,pj,'are candidates for Hadamard symmetry')
                    
                    # organize the remaining paulis to make detection of useful qubits easier
                    for i in range(1,P1.n_qudits()):
                        C, pivots = symplectic_reduction_iter_qudit_(P.copy(), C, [], i)
                    P1 = C.act(P.copy())
                    #print(P1)
                    #print()

                    # cancel the Y's in the first qubit
                    P1 = remove_Ys(P1)
                    #print()
                    #print(P2)
                    first_qubit_x = P2.x_exp[other_paulis,0]
                    #print(first_qubit_x%2)
                    if any(first_qubit_x%2):
                        print('No Hadamard Symmetry')
                    else:
                        return(P2)
                        # Erase phase
                        #           

In [185]:
start_pauli = PauliSum(['x1z0 x0z0', 'x0z1 x0z0','x0z0 x0z1','x0z0 x1z0', 'x1z1 x1z0'], dimensions=[2, 2], phases=[0,0,0,0,0],standardise=False)
C = add_r2(start_pauli,0,1)
print(C.act(start_pauli))

(1+0j)|x1z0 x0z1 | 0 
(1+0j)|x0z1 x0z1 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x1z1 x1z0 | 0 
(1+0j)|x0z0 x1z0 | 0 

